In [1]:
%matplotlib inline

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

# from konlpy.tag import Komoran; tokenizer = Komoran();

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
corpus_name = "normal chatbot data"
corpus = os.path.join("data", corpus_name)
textfilename = "shuffled_data.txt"

def printLines(file, n=10):
    with open(file, 'r', encoding = 'utf-8') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, textfilename))
datafile = os.path.join(corpus, textfilename)

아내가 이제 건강해져서 여행도 다니고 있어. 그런 모습을 보는 것이 참 기뻐.	아내가 건강해져서 여행도 다니실 수 있게 되다니 참 다행이에요.

그냥 공무원 친구들의 노력을 생각하면서 인정해주려고. 걔들이 노력한 만큼의 대가를 받는 거겠지 뭐.	당신의 생각대로 잘 돼서 질투심을 덜길 바라요.

생각도 못했는데 현명한 아내를 둬서 너무 고마워.	아내에게 고마운 마음을 어떻게 전하면 좋을까요?

오래 사귄 여자 친구와는 처음 만날 때처럼 떨림은 없지만 편안한 안정감을 느낄 수 있어.	오래 사귄 여자친구에게 편안한 안정감을 느껴서 행복하시군요!

응. 너무 편하다. 다음 달 대출 상황도 연체되지 않게 열심히 일해야겠어.	열심히 일해서 다음 달 대출 상환도 기한을 지키길 바라요.

다 내 잘못이지	아니에요. 너무 자책하지 마세요.

착해서 잘해주는 건지 좋아하는 건지	헷갈린다고 말해보세요.

선생님께 따돌림 사실을 진작 알렸다면 이렇게까지 심각해지지 않았을지도 모를 것 같아. 요새 너무 슬퍼.	일찍 선생님께 말씀 드리지 못한 것이 후회되시나 봐요.

나이를 먹으면서 몸 관리를 소홀하게 했더니 이렇게 빨리 늙어버렸네.	다른 일로 바쁘셨나 봐요. 지금이라도 운동하며 관리 하면 어떨까요?

주말은 근무시간이 아닌데 자꾸 회사 사람들이 불러내서 짜증나.	주말에 회사 사람들 때문에 쉬질 못하여서 화가 나셨군요. 조금 더 자세히 말씀해주시겠어요?



In [4]:
# 기본 단어 토큰 값
PAD_token = 0  # 짧은 문장을 채울(패딩, PADding) 때 사용할 제로 토큰
SOS_token = 1  # 문장의 시작(SOS, Start Of Sentence)을 나타내는 토큰
EOS_token = 2  # 문장의 끝(EOS, End Of Sentence)을 나태는 토큰

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # SOS, EOS, PAD를 센 것

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # 등장 횟수가 기준 이하인 단어를 정리합니다
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # 사전을 다시 초기화힙니다
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # 기본 토큰을 센 것

        for word in keep_words:
            self.addWord(word)

In [5]:
MAX_LENGTH = 20  # 고려할 문장의 최대 길이

# 유니코드 문자열을 아스키로 변환합니다
# https://stackoverflow.com/a/518232/2809427 참고
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 소문자로 만들고, 공백을 넣고, 알파벳 외의 글자를 제거합니다
def normalizeString(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣 ^☆; ^a-zA-Z.!?;0-9]+')
    result = hangul.sub('', s)
    return result

# 질의/응답 쌍을 읽어서 voc 객체를 반환합니다
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # 파일을 읽고, 쪼개어 lines에 저장합니다
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # 각 줄을 쪼개어 pairs에 저장하고 정규화합니다
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# 문장의 쌍 'p'에 포함된 두 문장이 모두 MAX_LENGTH라는 기준보다 짧은지를 반환합니다
def filterPair(p):
    # EOS 토큰을 위해 입력 시퀀스의 마지막 단어를 보존해야 합니다
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# 조건식 filterPair에 따라 pairs를 필터링합니다
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# 앞에서 정의한 함수를 이용하여 만든 voc 객체와 리스트 pairs를 반환합니다
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# voc와 pairs를 읽고 재구성합니다
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# 검증을 위해 pairs의 일부 내용을 출력해 봅니다
print("\npairs:")
for pair in pairs[:10]:

    print(pair)

Start preparing training data ...
Reading lines...
Read 37846 sentence pairs
Trimmed to 37804 sentence pairs
Counting words...
Counted words: 66491

pairs:
['아내가 이제 건강해져서 여행도 다니고 있어. 그런 모습을 보는 것이 참 기뻐.', '아내가 건강해져서 여행도 다니실 수 있게 되다니 참 다행이에요.']
['그냥 공무원 친구들의 노력을 생각하면서 인정해주려고. 걔들이 노력한 만큼의 대가를 받는 거겠지 뭐.', '당신의 생각대로 잘 돼서 질투심을 덜길 바라요.']
['생각도 못했는데 현명한 아내를 둬서 너무 고마워.', '아내에게 고마운 마음을 어떻게 전하면 좋을까요?']
['오래 사귄 여자 친구와는 처음 만날 때처럼 떨림은 없지만 편안한 안정감을 느낄 수 있어.', '오래 사귄 여자친구에게 편안한 안정감을 느껴서 행복하시군요!']
['응. 너무 편하다. 다음 달 대출 상황도 연체되지 않게 열심히 일해야겠어.', '열심히 일해서 다음 달 대출 상환도 기한을 지키길 바라요.']
['다 내 잘못이지', '아니에요. 너무 자책하지 마세요.']
['착해서 잘해주는 건지 좋아하는 건지', '헷갈린다고 말해보세요.']
['선생님께 따돌림 사실을 진작 알렸다면 이렇게까지 심각해지지 않았을지도 모를 것 같아. 요새 너무 슬퍼.', '일찍 선생님께 말씀 드리지 못한 것이 후회되시나 봐요.']
['나이를 먹으면서 몸 관리를 소홀하게 했더니 이렇게 빨리 늙어버렸네.', '다른 일로 바쁘셨나 봐요. 지금이라도 운동하며 관리 하면 어떨까요?']
['주말은 근무시간이 아닌데 자꾸 회사 사람들이 불러내서 짜증나.', '주말에 회사 사람들 때문에 쉬질 못하여서 화가 나셨군요. 조금 더 자세히 말씀해주시겠어요?']


In [6]:
MIN_COUNT = 2    # 제외할 단어의 기준이 되는 등장 횟수

def trimRareWords(voc, pairs, MIN_COUNT):
    # MIN_COUNT 미만으로 사용된 단어는 voc에서 제외합니다
    voc.trim(MIN_COUNT)
    # 제외할 단어가 포함된 경우를 pairs에서도 제외합니다
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # 입력 문장을 검사합니다
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # 출력 문장을 검사합니다
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # 입출력 문장에 제외하기로 한 단어를 포함하지 않는 경우만을 남겨둡니다
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# voc와 pairs를 정돈합니다
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 29729 / 66488 = 0.4471
Trimmed from 37804 pairs to 16058, 0.4248 of total


In [7]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# 입력 시퀀스 텐서에 패딩한 결과와 lengths를 반환합니다
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# 패딩한 목표 시퀀스 텐서, 패딩 마스크, 그리고 최대 목표 길이를 반환합니다
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# 입력 배치를 이루는 쌍에 대한 모든 아이템을 반환합니다
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # GRU를 초기화합니다. input_size와 hidden_size 패러미터는 둘 다 'hidden_size'로
        # 둡니다. 이는 우리 입력의 크기가 hideen_size 만큼의 피처를 갖는 단어 임베딩이기
        # 때문입니다.
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # 단어 인덱스를 임베딩으로 변환합니다
        embedded = self.embedding(input_seq)
        # RNN 모듈을 위한 패딩된 배치 시퀀스를 패킹합니다
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        # GRU로 포워드 패스를 수행합니다
        outputs, hidden = self.gru(packed, hidden)
        # 패딩을 언패킹합니다
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # 양방향 GRU의 출력을 합산합니다
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # 출력과 마지막 은닉 상태를 반환합니다
        return outputs, hidden

In [9]:
# Luong 어텐션 레이어
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Attention 가중치(에너지)를 제안된 방법에 따라 계산합니다
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # max_length와 batch_size의 차원을 뒤집습니다
        attn_energies = attn_energies.t()

        # 정규화된 softmax 확률 점수를 반환합니다 (차원을 늘려서)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [10]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 참조를 보존해 둡니다
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 레이어를 정의합니다
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 주의: 한 단위 시간에 대해 한 단계(단어)만을 수행합니다
        # 현재의 입력 단어에 대한 임베딩을 구합니다
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # 무방향 GRU로 포워드 패스를 수행합니다
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 현재의 GRU 출력을 바탕으로 어텐션 가중치를 계산합니다
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # 인코더 출력에 어텐션을 곱하여 새로운 "가중치 합" 문백 벡터를 구합니다
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Luong의 논문에 나온 식 5를 이용하여 가중치 문백 벡터와 GRU 출력을 결합합니다
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Luong의 논문에 나온 식 6을 이용하여 다음 단어를 예측합니다
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # 출력과 마지막 은닉 상태를 반환합니다
        return output, hidden

In [11]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [12]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # 제로 그라디언트
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # device 옵션을 설정합니다
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # 변수를 초기화합니다
    loss = 0
    print_losses = []
    n_totals = 0

    # 인코더로 포워드 패스를 수행합니다
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # 초기 디코더 입력을 생성합니다(각 문장을 SOS 도큰으로 시작합니다)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # 디코더의 초기 은닉 상태를 인코더의 마지막 은닉 상태로 둡니다
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # 이번 반복에서 teacher forcing을 사용할지를 결정합니다
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # 배치 시퀀스를 한 번에 하나씩 디코더로 포워드 패스합니다
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing 사용: 다음 입력을 현재의 목표로 둡니다
            decoder_input = target_variable[t].view(1, -1)
            # 손실을 계산하고 누적합니다
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing 미사용: 다음 입력을 디코더의 출력으로 둡니다
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # 손실을 계산하고 누적합니다
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # 역전파를 수행합니다
    loss.backward()

    # 그라디언트 클리핑: 그라디언트를 제자리에서 수정합니다
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # 모델의 가중치를 수정합니다
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [13]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 각 단계에 대한 배치를 읽어옵니다
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # 초기화
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # 학습 루프
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # 배치에서 각 필드를 읽어옵니다
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # 배치에 대해 학습을 한 단계 진행합니다
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # 경과를 출력합니다
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Checkpoint를 저장합니다
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [14]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # 인코더 모델로 입력을 포워드 패스합니다
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # 인코더의 마지막 은닉 레이어가 디코더의 첫 번째 은닉 레이어의 입력이 되도록 준비합니다
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # 디코더의 첫 번째 입력을 SOS_token으로 초기화합니다
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # 디코더가 단어를 덧붙여 나갈 텐서를 초기화합니다
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # 반복적으로 각 단계마다 하나의 단어 토큰을 디코딩합니다
        for _ in range(max_length):
            # 디코더로의 포워드 패스를 수행합니다
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # 가장 가능성 높은 단어 토큰과 그 softmax 점수를 구합니다
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # 토큰과 점수를 기록합니다
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # 현재의 토큰을 디코더의 다음 입력으로 준비시킵니다(차원을 증가시켜서)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # 단어 토큰과 점수를 모아서 반환합니다
        return all_tokens, all_scores

In [15]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### 입력 시퀀스를 배치 형태로 만듭니다
    # 단어 -> 인덱스
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # lengths 텐서를 만듭니다
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # 배치의 차원을 뒤집어서 모델이 사용하는 형태로 만듭니다
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 적절한 디바이스를 사용합니다
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # searcher를 이용하여 문장을 디코딩합니다
    tokens, scores = searcher(input_batch, lengths, max_length)
    # 인덱스 -> 단어
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 입력 문장을 받아옵니다
            input_sentence = input('> ')
            # 종료 조건인지 검사합니다
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 문장을 정규화합니다
            input_sentence = normalizeString(input_sentence)
            # 문장을 평가합니다
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 응답 문장을 형식에 맞춰 출력합니다
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: 아직 학습중이라 똑똑하지 않아요. 다른 문장을 입력해주세요!")

In [16]:
# 모델을 설정합니다
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 512
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# 불러올 checkpoint를 설정합니다. 처음부터 시작할 때는 None으로 둡니다.
loadFilename = None
checkpoint_iter = 5000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# loadFilename이 제공되는 경우에는 모델을 불러옵니다
if loadFilename:
    # 모델을 학습할 때와 같은 기기에서 불러오는 경우
    checkpoint = torch.load(loadFilename)
    # GPU에서 학습한 모델을 CPU로 불러오는 경우
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# 단어 임베딩을 초기화합니다
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# 인코더 및 디코더 모델을 초기화합니다
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# 적절한 디바이스를 사용합니다
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [17]:
# 학습 및 최적화 설정
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 10000
print_every = 1
save_every = 500

# Dropout 레이어를 학습 모드로 둡니다
encoder.train()
decoder.train()

# Optimizer를 초기화합니다
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# cuda가 있다면 cuda를 설정합니다
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
    
# 학습 단계를 수행합니다
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 10.3036
Iteration: 2; Percent complete: 0.0%; Average loss: 10.2698
Iteration: 3; Percent complete: 0.0%; Average loss: 10.2217
Iteration: 4; Percent complete: 0.0%; Average loss: 10.1423
Iteration: 5; Percent complete: 0.1%; Average loss: 10.0072
Iteration: 6; Percent complete: 0.1%; Average loss: 9.7348
Iteration: 7; Percent complete: 0.1%; Average loss: 9.3329
Iteration: 8; Percent complete: 0.1%; Average loss: 8.9941
Iteration: 9; Percent complete: 0.1%; Average loss: 8.7748
Iteration: 10; Percent complete: 0.1%; Average loss: 8.9106
Iteration: 11; Percent complete: 0.1%; Average loss: 8.9289
Iteration: 12; Percent complete: 0.1%; Average loss: 9.2281
Iteration: 13; Percent complete: 0.1%; Average loss: 9.1737
Iteration: 14; Percent complete: 0.1%; Average loss: 9.0971
Iteration: 15; Percent complete: 0.1%; Average loss: 8.6187
Iteration: 16; Percent complete:

Iteration: 136; Percent complete: 1.4%; Average loss: 5.9847
Iteration: 137; Percent complete: 1.4%; Average loss: 6.6702
Iteration: 138; Percent complete: 1.4%; Average loss: 6.6982
Iteration: 139; Percent complete: 1.4%; Average loss: 6.3198
Iteration: 140; Percent complete: 1.4%; Average loss: 6.3982
Iteration: 141; Percent complete: 1.4%; Average loss: 6.4282
Iteration: 142; Percent complete: 1.4%; Average loss: 6.1874
Iteration: 143; Percent complete: 1.4%; Average loss: 6.3460
Iteration: 144; Percent complete: 1.4%; Average loss: 6.4016
Iteration: 145; Percent complete: 1.5%; Average loss: 6.4880
Iteration: 146; Percent complete: 1.5%; Average loss: 6.1052
Iteration: 147; Percent complete: 1.5%; Average loss: 6.2736
Iteration: 148; Percent complete: 1.5%; Average loss: 6.1753
Iteration: 149; Percent complete: 1.5%; Average loss: 6.4328
Iteration: 150; Percent complete: 1.5%; Average loss: 6.0795
Iteration: 151; Percent complete: 1.5%; Average loss: 6.6879
Iteration: 152; Percent 

Iteration: 271; Percent complete: 2.7%; Average loss: 5.9037
Iteration: 272; Percent complete: 2.7%; Average loss: 5.9806
Iteration: 273; Percent complete: 2.7%; Average loss: 5.6919
Iteration: 274; Percent complete: 2.7%; Average loss: 6.0322
Iteration: 275; Percent complete: 2.8%; Average loss: 5.9056
Iteration: 276; Percent complete: 2.8%; Average loss: 5.5141
Iteration: 277; Percent complete: 2.8%; Average loss: 5.6125
Iteration: 278; Percent complete: 2.8%; Average loss: 5.5609
Iteration: 279; Percent complete: 2.8%; Average loss: 5.4230
Iteration: 280; Percent complete: 2.8%; Average loss: 5.7956
Iteration: 281; Percent complete: 2.8%; Average loss: 5.3510
Iteration: 282; Percent complete: 2.8%; Average loss: 5.8567
Iteration: 283; Percent complete: 2.8%; Average loss: 5.9930
Iteration: 284; Percent complete: 2.8%; Average loss: 5.5506
Iteration: 285; Percent complete: 2.9%; Average loss: 5.8474
Iteration: 286; Percent complete: 2.9%; Average loss: 5.2400
Iteration: 287; Percent 

Iteration: 406; Percent complete: 4.1%; Average loss: 5.2341
Iteration: 407; Percent complete: 4.1%; Average loss: 5.2094
Iteration: 408; Percent complete: 4.1%; Average loss: 5.0628
Iteration: 409; Percent complete: 4.1%; Average loss: 5.3750
Iteration: 410; Percent complete: 4.1%; Average loss: 5.5962
Iteration: 411; Percent complete: 4.1%; Average loss: 5.5042
Iteration: 412; Percent complete: 4.1%; Average loss: 5.0435
Iteration: 413; Percent complete: 4.1%; Average loss: 5.6601
Iteration: 414; Percent complete: 4.1%; Average loss: 5.8170
Iteration: 415; Percent complete: 4.2%; Average loss: 5.2489
Iteration: 416; Percent complete: 4.2%; Average loss: 5.4276
Iteration: 417; Percent complete: 4.2%; Average loss: 5.3705
Iteration: 418; Percent complete: 4.2%; Average loss: 5.3130
Iteration: 419; Percent complete: 4.2%; Average loss: 5.4290
Iteration: 420; Percent complete: 4.2%; Average loss: 5.4635
Iteration: 421; Percent complete: 4.2%; Average loss: 5.3077
Iteration: 422; Percent 

Iteration: 541; Percent complete: 5.4%; Average loss: 5.3597
Iteration: 542; Percent complete: 5.4%; Average loss: 5.1737
Iteration: 543; Percent complete: 5.4%; Average loss: 4.8296
Iteration: 544; Percent complete: 5.4%; Average loss: 5.3752
Iteration: 545; Percent complete: 5.5%; Average loss: 4.8048
Iteration: 546; Percent complete: 5.5%; Average loss: 4.9405
Iteration: 547; Percent complete: 5.5%; Average loss: 5.0276
Iteration: 548; Percent complete: 5.5%; Average loss: 4.7856
Iteration: 549; Percent complete: 5.5%; Average loss: 4.7580
Iteration: 550; Percent complete: 5.5%; Average loss: 4.4395
Iteration: 551; Percent complete: 5.5%; Average loss: 4.6110
Iteration: 552; Percent complete: 5.5%; Average loss: 4.6018
Iteration: 553; Percent complete: 5.5%; Average loss: 5.2944
Iteration: 554; Percent complete: 5.5%; Average loss: 5.1028
Iteration: 555; Percent complete: 5.5%; Average loss: 5.2904
Iteration: 556; Percent complete: 5.6%; Average loss: 4.8910
Iteration: 557; Percent 

...

Iteration: 9462; Percent complete: 94.6%; Average loss: 0.0616
Iteration: 9463; Percent complete: 94.6%; Average loss: 0.0524
Iteration: 9464; Percent complete: 94.6%; Average loss: 0.0720
Iteration: 9465; Percent complete: 94.7%; Average loss: 0.0554
Iteration: 9466; Percent complete: 94.7%; Average loss: 0.1005
Iteration: 9467; Percent complete: 94.7%; Average loss: 0.0460
Iteration: 9468; Percent complete: 94.7%; Average loss: 0.0671
Iteration: 9469; Percent complete: 94.7%; Average loss: 0.0466
Iteration: 9470; Percent complete: 94.7%; Average loss: 0.0652
Iteration: 9471; Percent complete: 94.7%; Average loss: 0.0437
Iteration: 9472; Percent complete: 94.7%; Average loss: 0.0455
Iteration: 9473; Percent complete: 94.7%; Average loss: 0.0847
Iteration: 9474; Percent complete: 94.7%; Average loss: 0.0943
Iteration: 9475; Percent complete: 94.8%; Average loss: 0.0277
Iteration: 9476; Percent complete: 94.8%; Average loss: 0.0234
Iteration: 9477; Percent complete: 94.8%; Average loss:

Iteration: 9593; Percent complete: 95.9%; Average loss: 0.0579
Iteration: 9594; Percent complete: 95.9%; Average loss: 0.0328
Iteration: 9595; Percent complete: 96.0%; Average loss: 0.0461
Iteration: 9596; Percent complete: 96.0%; Average loss: 0.0529
Iteration: 9597; Percent complete: 96.0%; Average loss: 0.0580
Iteration: 9598; Percent complete: 96.0%; Average loss: 0.0454
Iteration: 9599; Percent complete: 96.0%; Average loss: 0.0514
Iteration: 9600; Percent complete: 96.0%; Average loss: 0.0752
Iteration: 9601; Percent complete: 96.0%; Average loss: 0.0553
Iteration: 9602; Percent complete: 96.0%; Average loss: 0.0952
Iteration: 9603; Percent complete: 96.0%; Average loss: 0.0540
Iteration: 9604; Percent complete: 96.0%; Average loss: 0.1002
Iteration: 9605; Percent complete: 96.0%; Average loss: 0.0256
Iteration: 9606; Percent complete: 96.1%; Average loss: 0.0498
Iteration: 9607; Percent complete: 96.1%; Average loss: 0.1289
Iteration: 9608; Percent complete: 96.1%; Average loss:

Iteration: 9724; Percent complete: 97.2%; Average loss: 0.0789
Iteration: 9725; Percent complete: 97.2%; Average loss: 0.0641
Iteration: 9726; Percent complete: 97.3%; Average loss: 0.0567
Iteration: 9727; Percent complete: 97.3%; Average loss: 0.0616
Iteration: 9728; Percent complete: 97.3%; Average loss: 0.0516
Iteration: 9729; Percent complete: 97.3%; Average loss: 0.0430
Iteration: 9730; Percent complete: 97.3%; Average loss: 0.0735
Iteration: 9731; Percent complete: 97.3%; Average loss: 0.0722
Iteration: 9732; Percent complete: 97.3%; Average loss: 0.0432
Iteration: 9733; Percent complete: 97.3%; Average loss: 0.0364
Iteration: 9734; Percent complete: 97.3%; Average loss: 0.0671
Iteration: 9735; Percent complete: 97.4%; Average loss: 0.0297
Iteration: 9736; Percent complete: 97.4%; Average loss: 0.0239
Iteration: 9737; Percent complete: 97.4%; Average loss: 0.0800
Iteration: 9738; Percent complete: 97.4%; Average loss: 0.0513
Iteration: 9739; Percent complete: 97.4%; Average loss:

Iteration: 9855; Percent complete: 98.6%; Average loss: 0.0555
Iteration: 9856; Percent complete: 98.6%; Average loss: 0.0756
Iteration: 9857; Percent complete: 98.6%; Average loss: 0.0866
Iteration: 9858; Percent complete: 98.6%; Average loss: 0.0478
Iteration: 9859; Percent complete: 98.6%; Average loss: 0.0860
Iteration: 9860; Percent complete: 98.6%; Average loss: 0.0920
Iteration: 9861; Percent complete: 98.6%; Average loss: 0.0680
Iteration: 9862; Percent complete: 98.6%; Average loss: 0.0917
Iteration: 9863; Percent complete: 98.6%; Average loss: 0.0529
Iteration: 9864; Percent complete: 98.6%; Average loss: 0.0565
Iteration: 9865; Percent complete: 98.7%; Average loss: 0.0656
Iteration: 9866; Percent complete: 98.7%; Average loss: 0.0907
Iteration: 9867; Percent complete: 98.7%; Average loss: 0.0929
Iteration: 9868; Percent complete: 98.7%; Average loss: 0.0665
Iteration: 9869; Percent complete: 98.7%; Average loss: 0.0693
Iteration: 9870; Percent complete: 98.7%; Average loss:

Iteration: 9986; Percent complete: 99.9%; Average loss: 0.0676
Iteration: 9987; Percent complete: 99.9%; Average loss: 0.0693
Iteration: 9988; Percent complete: 99.9%; Average loss: 0.1121
Iteration: 9989; Percent complete: 99.9%; Average loss: 0.0576
Iteration: 9990; Percent complete: 99.9%; Average loss: 0.0464
Iteration: 9991; Percent complete: 99.9%; Average loss: 0.0538
Iteration: 9992; Percent complete: 99.9%; Average loss: 0.0443
Iteration: 9993; Percent complete: 99.9%; Average loss: 0.0816
Iteration: 9994; Percent complete: 99.9%; Average loss: 0.0430
Iteration: 9995; Percent complete: 100.0%; Average loss: 0.0753
Iteration: 9996; Percent complete: 100.0%; Average loss: 0.0657
Iteration: 9997; Percent complete: 100.0%; Average loss: 0.0458
Iteration: 9998; Percent complete: 100.0%; Average loss: 0.0575
Iteration: 9999; Percent complete: 100.0%; Average loss: 0.0685
Iteration: 10000; Percent complete: 100.0%; Average loss: 0.0623


In [ ]:
# Dropout 레이어를 평가 모드로 설정합니다
encoder.eval()
decoder.eval()

# 탐색 모듈을 초기화합니다
searcher = GreedySearchDecoder(encoder, decoder)

# 채팅을 시작합니다 (다음 줄의 주석을 제거하면 시작해볼 수 있습니다)
evaluateInput(encoder, decoder, searcher, voc)

> 안녕하세요.
Bot: 서로의 마음이 중요하겠죠. 어떻게 하면 좋을까요? 마음 편해질까요? 어떤 것이 좋을까요? 어떤 것이 좋을까요?
> 만나서 반가워요.
Bot: 자주 만나세요. 좋아요. 저랑 풀렸으면 좋겠어요. 저랑 알려주세요. 저랑 거예요. 저랑 풀렸으면 좋겠어요. 저랑
> 어디에 있나요?
Bot: 그럼요. 못했던 해보세요. 더 이상 기회를 해보세요. 어떤 방법이 있을까요? 어떤 것이 있을까요?
> 이름이 뭐예요?
Error: 아직 학습중이라 똑똑하지 않아요. 다른 문장을 입력해주세요!
> 집에 가고 싶다.
Bot: 그게 제일 좋을 것 같아요. 다른 사람이에요. 생각하고 수도 있어요. 어떤 방법이 있을까요? 어떤
> 집에 가고 싶네요.
Bot: 집이 쉬고 돼요. 조금만 드세요 조금만 해보세요. 어떤 곳이 있을까요? 어떤 곳이 있을까요?
> 놀러 가고 싶다.
Bot: 저도 잘 모르겠어요. 어떤 마음이 사라지면 까요? 어떤 것들이 있을까요? 좋을 것
> 놀러가고싶다.
Error: 아직 학습중이라 똑똑하지 않아요. 다른 문장을 입력해주세요!
> 좋은 아침!
Error: 아직 학습중이라 똑똑하지 않아요. 다른 문장을 입력해주세요!
> 좋은 아침입니다.
Error: 아직 학습중이라 똑똑하지 않아요. 다른 문장을 입력해주세요!
> 안녕하세요?
Bot: 좋은 소식이네요. 그게 분도 좋아할 거예요. 좋아할 거예요. 어떻게 하는 게 좋을까요? 어떻게
